**Imports**

In [1]:
!pip install --upgrade pip
!pip install sortedcontainers
!pip install returns

In [2]:
import sqlite3
import typing
import sortedcontainers
import enum
from returns import result

**Read file**

In [3]:
DATABASE_NAME = 'packing_trace_zone_a_v1.sqlite'

In [4]:
def read_sqlite_table(db_filename, table_name):
    try:
        sqlite_connection= sqlite3.connect(db_filename, timeout=20)
        cursor = sqlite_connection.cursor()
        print("Подключен к SQLite")
        sqlite_select_query = f"SELECT * from {table_name}"
        cursor.execute(sqlite_select_query)
        rows = cursor.fetchall()
        print("Всего строк: ", len(rows))
        cursor.close()
    except sqlite3.Error as error:
        print("Ошибка при подключении к sqlite", error)
    finally:
        if (sqlite_connection):
            sqlite_connection.close()
            print("Соединение с SQLite закрыто")
    return rows

In [5]:
vm_requests_data = read_sqlite_table(DATABASE_NAME, "vm")
vm_types_data = read_sqlite_table(DATABASE_NAME, "vmType")

Подключен к SQLite
Всего строк:  5559800
Соединение с SQLite закрыто
Подключен к SQLite
Всего строк:  4619
Соединение с SQLite закрыто


VM Requests:  
Field — Description  
vmId — unique id of the vm request  
tenantId — unique id for the owner of a group of requests  
vmTypeId — requested VM type  
priority — priority of the VM request  
starttime — the time (in fractional days) when the VM request was created  
endtime\tthe — time (in fractional days) when the VM left the system  

In [6]:
vm_requests_data[:10]

[(0, 0, 8, 0, -138.92548611108214, None),
 (1, 0, 8, 0, -138.92548611108214, 18.035496724769473),
 (2, 0, 8, 0, -138.92548611108214, 29.82878737291321),
 (3, 0, 8, 0, -138.92548611108214, None),
 (4, 0, 8, 0, -63.68957175919786, 53.58965337974951),
 (5, 0, 8, 0, 0.47931712958961725, 0.8728222916834056),
 (6, 0, 8, 0, -120.9216435183771, 8.964727916754782),
 (7, 0, 8, 0, -0.6837962963618338, 2.214773680549115),
 (8, 0, 8, 0, -120.91707175923511, 28.84202907420695),
 (9, 0, 8, 0, -0.7317129629664123, 1.0297179860062897)]

VM Types:  
Field — Description  
id — unique row id  
vmTypeId — unique id for VM type name, can be used on different machines  
machineId — unique id for machine  \n",
core — requested CPU resource allocation for this VM type on this machine  
memory — requested memory resource allocation for this VM type on this machine  
hdd — requested hard drive resource allocation for this VM type on this machine  
ssd — requested solid state drive resource allocation for this VM type on this machine  
nic — requested network bandwidth allocation for this VM type on this machine"

In [7]:
vm_types_data[:5]

[(1,
  0,
  0,
  0.020833333333333332,
  0.00390625,
  None,
  0.010064299692479732,
  0.000125),
 (2,
  1,
  0,
  0.020833333333333332,
  0.009114583333333334,
  None,
  0.013419066256639642,
  0.0025),
 (3,
  2,
  0,
  0.041666666666666664,
  0.018229166666666668,
  None,
  0.026838132513279284,
  0.005),
 (4,
  3,
  0,
  0.08333333333333333,
  0.036458333333333336,
  None,
  0.05367626502655857,
  0.01),
 (5,
  4,
  0,
  0.16666666666666666,
  0.07291666666666667,
  None,
  0.10735253005311714,
  0.02)]

**Library**

In [8]:
class VirtualMachine:
    def __init__(self, vm_type_id: int, machine_id: int, core: float, memory: float, hdd: typing.Optional[float], ssd: float, 
                 nic: float):
        self.vm_type_id: int = vm_type_id
        self.machine_id: int = machine_id
        self.core: float = core
        self.memory: float = memory
        self.hdd: float = hdd
        self.ssd: float = ssd
        self.nic: float = nic
        self.active: bool = True
        self.available: bool = True
        
    def destroy(self):
        self.active = False

In [9]:
class Request:
        relevant_machine: typing.Dict[int, VirtualMachine] = {}
        
        class Type(enum.Enum):
            START = 0
            END = 1
        
        def __init__(self, request_id: int, tenant_id: int, vm_type_id: int, priority: int, start_time: float,
                     end_time: typing.Optional[float], request_type: Type):
            self.request_id: int = request_id
            self.request_type: Type = request_type
            self.tenant_id: int = tenant_id
            self.vm_type_id: int = vm_type_id
            self.priority: int = priority
            self.start_time: float = start_time
            self.end_time: float = end_time
            self.__delay: float = None
            self.retries: int = 0
            self.result: result.Result = None
        
        @property
        def delay(self) -> float:
            if not self.__delay:
                raise ReferenceError("Request has not been completed yet. Try again later.")
            return self.__delay
        
        @property
        def machine(self) -> VirtualMachine:
            if self.request_id not in Request.relevant_machine:
                return None
            return Request.relevant_machine[self.request_id]
        
        def close(self):
            if not self.machine:
                self.result = result.Failure("Request has not been allocated untill the end.")
                return
            self.machine.available = True
            del Request.relevant_machine[self.request_id]
            request.result = result.Success("Request has been closed successfully.")
        
        def assign_machine(self, machine: VirtualMachine):
            Request.relevant_machine[self.request_id] = machine
            machine.available = True
            self.__delay = self.__comparison_time - self.start_time
        
        def fit(self, machine: VirtualMachine) -> bool:
            return self.vm_type_id == machine.vm_type_id
        
        @property
        def __comparison_time(self) -> float:
            return self.start_time if self.request_type == Request.Type.START else self.end_time
            
        def __lt__(self, other) -> bool:
            return self.__comparison_time < other.__comparison_time
        
        def __lt__(self, other) -> bool:
            return self.__comparison_time == other.__comparison_time

In [10]:
class Distributor:
        def __init__(self, max_retries: int):
            self.__machines: typing.Iterable[VirtualMachine] = []
            self.__active_requests: typing.Set[typing.Tuple[int, Request]] = sortedcontainers.SortedSet()
            self.max_retries = max_retries
        
        def add_machine(self, machine: VirtualMachine):
            self.__machines.append(machine)
        
        def add_bulk_machines(self, machines: typing.Iterable[VirtualMachine]):
            self.__machines.extend(machines)
        
        def assign_machine_to_request(self, request: Request):
            if request.retries > self.max_retries:
                request.result = result.Failure("Too many retries.")
            else:
                request.retries += 1
            
            for machine in self.__machines:
                if machine.available and request.fit(machine):
                    request.assign_machine(machine)
                    request.result = result.Success("Machine has been assigned.")
                    break
            else:
                request.result = result.Failure("There are no suitable machines.")
            return request.result
        
        def allocate_service(self, request: Request) -> result.Result:
            if request.request_type == Request.Type.END:
                request.close()
            else:
                self.__active_requests.add((request.priority, request))
            
            deleting_requests: typing.Iterable[typing.Tuple[int, Request]] = []
            for priority, old_request in reversed(self.__active_requests):
                if self.assign_machine_to_request(old_request):
                    deleting_requests.append((priority, old_request))
            
            for priority, old_request in deleting_requests:
                self.__active_requests.discard((priority, old_request))
            
            return request.result

In [11]:
test_distributor = Distributor(5)
test_request = Request(1, 0, 8, 0, -138.92548611108214, 18.035496724769473, Request.Type.START)
test_vm = VirtualMachine(0, 0, 0.020833333333333332, 0.00390625, None, 0.010064299692479732, 0.000125)
test_distributor.add_machine(test_vm)
test_distributor.allocate_service(test_request)

<Failure: There are no suitable machines.>

**Simulation**

In [12]:
distributor = Distributor(5)
for row in vm_types_data:
    res = distributor.add_machine(VirtualMachine(*row[1:]))

In [13]:
timeline: typing.Iterable[Request] = sortedcontainers.SortedList()
for row in vm_requests_data:
    res = timeline.add(Request(*row, Request.Type.START))
    if row[-1]:
        res = timeline.add(Request(*row, Request.Type.END))

In [14]:
for request in timeline:
    res = distributor.allocate_service(request)

In [15]:
delays = []
for request in timeline:
    try:
        delays.append(request.delay)
    except:
        continue

**Draw graph**